# US 2017 State, County, Minor Civil Division, and Incorporated Place FIPS Codes  

**[Work in progress]**

This notebook creates .csv files with US 2017 State and County FIPS codes for ingestion into the Knowledge Graph.

Data source: [2017 State, County, Minor Civil Division, and Incorporated Place FIPS Codes](https://www2.census.gov/programs-surveys/popest/geographies/2017/all-geocodes-v2017.xlsx)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
from pathlib import Path
import pandas as pd

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-19636412-9e74-4bac-8a4c-c6c8b49bb9d3/installation-4.1.0/import


In [4]:
census_url = 'https://www2.census.gov/programs-surveys/popest/geographies/2017/all-geocodes-v2017.xlsx'

In [5]:
df = pd.read_excel(census_url, dtype='str', skiprows=4)

In [6]:
df.head()

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
0,010,00,000,00000,00000,00000,United States
1,040,01,000,00000,00000,00000,Alabama
2,050,01,001,00000,00000,00000,Autauga County
3,050,01,003,00000,00000,00000,Baldwin County
4,050,01,005,00000,00000,00000,Barbour County


In [7]:
df.rename(columns={'Area Name (including legal/statistical area description)': 'name'}, inplace=True)

### Create list of US Counties

In [8]:
counties = df.query("`County Code (FIPS)` != '000'").query("`County Subdivision Code (FIPS)` == '00000'").copy()
counties.rename(columns={'County Code (FIPS)': 'fips'}, inplace=True)
counties['stateFips'] = counties['State Code (FIPS)']
counties = counties[['name', 'fips', 'stateFips']]
counties['geoId'] = counties['stateFips'] + counties['fips']

In [9]:
counties.count()

name         3220
fips         3220
stateFips    3220
geoId        3220
dtype: int64

In [10]:
counties.head()

,name,fips,stateFips,geoId
2,Autauga County,001,01,01001
3,Baldwin County,003,01,01003
4,Barbour County,005,01,01005
5,Bibb County,007,01,01007
6,Blount County,009,01,01009


In [11]:
counties.query("fips == '039'").query("stateFips == '01'")

,name,fips,stateFips,geoId
21,Covington County,039,01,01039


In [12]:
counties.query("geoId == '01101'")

,name,fips,stateFips,geoId
52,Montgomery County,101,01,01101


In [13]:
counties.to_csv(NEO4J_IMPORT / "00j-USCensus2017County.csv", index=False)

### Create list of US Cities

In [14]:
cities = df.query("`Place Code (FIPS)` != '00000'").copy()
cities.rename(columns={'Place Code (FIPS)': 'fips'}, inplace=True)
cities.rename(columns={'State Code (FIPS)': 'stateFips'}, inplace=True)
cities = cities[['name', 'fips', 'stateFips']]

In [15]:
cities.head()

,name,fips,stateFips
69,Abbeville city,00124,01
70,Adamsville city,00460,01
71,Addison town,00484,01
72,Akron town,00676,01
73,Alabaster city,00820,01


Remove lower case city or town, they are not part of the city name. This does not effect city names that have "City" as part of their name, e.g., Atlantic City.

In [16]:
cities['name'] = cities['name'].str.replace(' city','')
cities['name'] = cities['name'].str.replace(' town','')
cities.head()

,name,fips,stateFips
69,Abbeville,00124,01
70,Adamsville,00460,01
71,Addison,00484,01
72,Akron,00676,01
73,Alabaster,00820,01


In [17]:
cities.query("name == 'Atlantic City'")

,name,fips,stateFips
23490,Atlantic City,02080,34


In [18]:
cities.query("name == 'Covington'")

,name,fips,stateFips
3306,Covington,20064,13
8018,Covington,15490,18
11936,Covington,17848,21
12393,Covington,18125,22
31243,Covington,17700,40
37811,Covington,17680,47
38588,Covington,17372,48
40337,Covington,19728,51
40603,Covington,15290,53


In [19]:
cities.to_csv(NEO4J_IMPORT / "00j-USCensus2017City.csv", index=False)